## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Walvis Bay,NaN,-22.96,14.51,53.01,93,0,8.05,clear sky
1,1,Lagos,NG,6.58,3.75,75.49,94,100,2.75,overcast clouds
2,2,Qasigiannguit,GL,68.82,-51.19,46.40,49,100,16.11,overcast clouds
3,3,Alto Araguaia,BR,-17.31,-53.22,77.58,42,29,5.06,scattered clouds
4,4,Coquimbo,CL,-29.95,-71.34,53.60,87,0,2.24,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp =float(input("What do you want the max temp to be?"))
min_temp = float(input("What do you want the min temp to be?"))

What do you want the max temp to be?90
What do you want the min temp to be?75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
new_city_data_df=city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] <= min_temp)]
new_city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Walvis Bay,NaN,-22.96,14.51,53.01,93,0,8.05,clear sky
2,2,Qasigiannguit,GL,68.82,-51.19,46.40,49,100,16.11,overcast clouds
4,4,Coquimbo,CL,-29.95,-71.34,53.60,87,0,2.24,clear sky
6,6,Zaysan,KZ,47.47,84.87,41.59,55,90,2.17,overcast clouds
7,7,Cidreira,BR,-30.18,-50.21,57.65,72,0,7.90,clear sky
...,...,...,...,...,...,...,...,...,...,...
692,692,Shahrud,IR,36.42,54.98,45.61,48,0,2.59,clear sky
693,693,Aranda De Duero,ES,41.67,-3.69,44.62,71,0,11.59,clear sky
694,694,Hammerfest,NO,70.66,23.68,44.60,100,100,19.46,drizzle
695,695,Antofagasta,CL,-23.65,-70.40,59.00,72,90,5.82,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
new_city_data_df.isnull().sum()

City_ID                 0
City                    0
Country                10
Lat                     0
Lng                     0
Max Temp                0
Humidity                0
Cloudiness              0
Wind Speed              0
Current Description     0
dtype: int64

In [6]:
new_city_data_df.shape

(497, 10)

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = new_city_data_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Qasigiannguit,GL,46.40,overcast clouds,68.82,-51.19,
4,Coquimbo,CL,53.60,clear sky,-29.95,-71.34,
6,Zaysan,KZ,41.59,overcast clouds,47.47,84.87,
7,Cidreira,BR,57.65,clear sky,-30.18,-50.21,
8,Sheregesh,RU,31.05,overcast clouds,52.92,87.99,
12,Husavik,IS,30.20,overcast clouds,66.04,-17.34,
13,Hermanus,ZA,48.99,clear sky,-34.42,19.23,
14,Borsa,RO,51.01,light rain,47.66,24.66,
16,Lively,CA,39.00,few clouds,46.43,-81.15,
17,Port Alfred,ZA,53.73,light rain,-33.59,26.89,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]


    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"]= f"{lat}, {lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    #print(response.url)
    hotels = response.json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Grab the first hotel from the results and store the name.
    try:
       hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
# # 7. Drop the rows where there is no Hotel Name.
# hotel_df['Hotel Name'].replace("", np.nan, inplace=True)

In [11]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Qasigiannguit,GL,46.40,overcast clouds,68.82,-51.19,
4,Coquimbo,CL,53.60,clear sky,-29.95,-71.34,Hotel San Juan
6,Zaysan,KZ,41.59,overcast clouds,47.47,84.87,Gostinitsa Aray
7,Cidreira,BR,57.65,clear sky,-30.18,-50.21,Hotel Castelo
8,Sheregesh,RU,31.05,overcast clouds,52.92,87.99,Hotel Olymp
...,...,...,...,...,...,...,...
692,Shahrud,IR,45.61,clear sky,36.42,54.98,
693,Aranda De Duero,ES,44.62,clear sky,41.67,-3.69,
694,Hammerfest,NO,44.60,drizzle,70.66,23.68,
695,Antofagasta,CL,59.00,overcast clouds,-23.65,-70.40,


In [12]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [13]:

clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Qasigiannguit,GL,46.40,overcast clouds,68.82,-51.19,
4,Coquimbo,CL,53.60,clear sky,-29.95,-71.34,Hotel San Juan
6,Zaysan,KZ,41.59,overcast clouds,47.47,84.87,Gostinitsa Aray
7,Cidreira,BR,57.65,clear sky,-30.18,-50.21,Hotel Castelo
8,Sheregesh,RU,31.05,overcast clouds,52.92,87.99,Hotel Olymp
...,...,...,...,...,...,...,...
692,Shahrud,IR,45.61,clear sky,36.42,54.98,
693,Aranda De Duero,ES,44.62,clear sky,41.67,-3.69,
694,Hammerfest,NO,44.60,drizzle,70.66,23.68,
695,Antofagasta,CL,59.00,overcast clouds,-23.65,-70.40,


In [14]:
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))